In [1]:
import pandas as pd
import numpy as np
import datetime
from pyexcelerate import Workbook
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

In [2]:
# типизируем cтолбцы таблица,
# type_ к какому типу приводим, 
# mask по каком услову в столбце типизируем
def retype_multiindex(df , type_, mask, levelForFindMask):
    for col in df.columns:
        if col[levelForFindMask].find(mask)!=-1:
            df.loc[:,col] = df.loc[:,col].fillna(0).astype('int').astype(type_)
    return df

def retype_index(df, type_ , mask):
    for col in df.columns:
        try:
            if col.find(mask)!=-1:
                df.loc[:,col] = df.loc[:,col].fillna(0).astype(type_)
        except AttributeError:
            pass
        except ValueError:
            pass
    return df
    

# Упаковка справочника для insert vpr
def rev_to_dict(ind,find):
#     ind = ind.astype('str')
    df = pd.DataFrame(index=ind,data=find.values,columns=[find.name])
    df = df[(df.index!=0)|(pd.isnull(df.index)==True)]
    df = df.groupby(df.index).first()
    dictt =  df.to_dict('index')
    return dictt  


# Аналог впр
def insert_vpr(x,dictt,name):
    try:
        return dictt[x][name]
    except KeyError:
        return 0
    
# Собириает текущие данные
def get_current_data(data, global_index, current_month):
    index = [x for x in data.columns if x in global_index]
    index.extend([current_month])
    return data[index]


def save_pyex(path,df):
    wb = Workbook()
    wb.new_sheet('sheet1',data =  [df.columns.tolist(),]+df.values.tolist())
    wb.save(path) 


    
def dict_keys(mapping):
    cons = pd.DataFrame()
    for ind, col in enumerate(mapping.columns):
        try:
            frame_ = mapping.loc[:, (col[0],['Артикул','Часть'])]
            frame_.columns = ['Артикул','Часть']
            cons = cons.append(frame_)
        except KeyError:
            pass
        except ValueError:
            pass
        
    return cons.drop_duplicates()


# Установить тип продукции для 3 передела
def get_type_product(data):
    colName = 'Тип продукции'
    data[colName] = ''
    for art in data['Артикул'].unique():
        min_index = min(list(data[data['Артикул']==art].index)  ) 
        max_index  = max(list(data[data['Артикул']==art].index)  ) 
#         data.loc[min_index,colName] = 'готовая продукция'
        data.loc[max_index,colName] = 'основная продукция'
    return data


# Установка новых % в накладные расходы по упаковке
def input_proc_packaging(df,arr_priznak, date, filter_, df2,  arr_priznak1 ,filter_2):
    cols1 = ('downstream', 'Упак', arr_priznak1[0])
    cols2 = ('downstream', 'Упак', arr_priznak1[1])
    cols3 = ('downstream', 'Упак', arr_priznak1[2])
    cols_all = ('downstream', 'Упак', 'всего')
    
    all_cost = df2.loc[df2['Месяц']['Месяц']['Месяц']==current_month, (cols_all)].sum()
    
    d1 = df[df[arr_priznak[0]] == "да"][current_month].sum() / \
    df2[df2['Наименование']['Наименование']['Наименование']==filter_].loc[:,cols1].sum()
    
    d2 = df[df[arr_priznak[1]] == "да"][current_month].sum() / \
    df2[df2['Наименование']['Наименование']['Наименование']==filter_].loc[:,cols2].sum()
    
    d3 = df[df[arr_priznak[2]] == "да"][current_month].sum() / \
    df2[df2['Наименование']['Наименование']['Наименование']==filter_].loc[:,cols3].sum()
    
    df2.loc[df2['Наименование']['Наименование']['Наименование']==filter_2,(cols1)] = d1/(d1+d2+d3)
    df2.loc[df2['Наименование']['Наименование']['Наименование']==filter_2,(cols2)] = d2/(d1+d2+d3)
    df2.loc[df2['Наименование']['Наименование']['Наименование']==filter_2,(cols3)] = d3/(d1+d2+d3)
    
    #     Подставляем значения всего умноженные на эти проценты
    
    df2.loc[df2['Месяц']['Месяц']['Месяц']==current_month,(cols1)] = d1/(d1+d2+d3) * all_cost
    df2.loc[df2['Месяц']['Месяц']['Месяц']==current_month,(cols2)] = d2/(d1+d2+d3) * all_cost
    df2.loc[df2['Месяц']['Месяц']['Месяц']==current_month,(cols3)] = d3/(d1+d2+d3) * all_cost
    


    return df2

In [3]:
directory = 'C:\\Users\\Кедрун Никита\\PycharmProjects\\akz_\\'

In [4]:
current_month  = datetime.datetime(2021,6,30)
global_index = ['Артикул', 'Продукция', 'Номенклатура', 'Канал', 'Тип']

In [ ]:
# загрузка источников
sale_finished_products_UKPF =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='Продажи ГП')
balance_depot_stock_UKPF =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='Остатки')
adm_UKPF =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='Адм')
PP_UKPF =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='РР')
per_3_UKPF =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='3 передел')
per_2_UKPF_meat =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='2 передел мясо')
per_2_UKPF_product =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='2 передел гп')
per_2_UKPF_losses =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='2 передел потери')
per_1_UKPF =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='1 передел')
NZ_UKPF =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='Накладные затраты',header=[0,1,2])
GP_UKPF =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='Выпуск ГП')
UIS_UKPF =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='УиС')
DATA_upload =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='Общие данные по выходу')
Production_output =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='Выпуск ГП')

In [ ]:
# Сквозные источники

mapping =  pd.read_excel(directory+'Mapping.xlsx',sheet_name='Mapping',header=[0,1] )
koef_ef =  pd.read_excel(directory+'Коэффициент ценности.xlsx',sheet_name='Лист1')
ost_start =  pd.read_excel(directory+'Остатки на начало года.xlsx',sheet_name='ОСТ УКПФ ДЕК',header=[0,1])

In [ ]:
# Блок типизации данных
dates = [x for x in PP_UKPF.columns if isinstance( x, datetime.datetime)]  

# Изменил тип артикула
mapping  = retype_multiindex(mapping, 'int', 'Артикул', 1)


# Изменил тип артикула

per_1_UKPF  = retype_index(per_1_UKPF, 'int', 'Артикул')
per_2_UKPF_meat  = retype_index(per_2_UKPF_meat, 'int', 'Артикул')
per_2_UKPF_product  = retype_index(per_2_UKPF_product, 'int', 'Артикул')
per_2_UKPF_losses  = retype_index(per_2_UKPF_losses, 'int', 'Артикул')
per_3_UKPF  = retype_index(per_3_UKPF, 'int', 'Артикул')
Production_output  = retype_index(Production_output, 'int', 'Артикул')
UIS_UKPF  = retype_index(UIS_UKPF, 'int', 'Артикул')
ost_start  = retype_multiindex(ost_start, 'int', 'Артикул', 1)
sale_finished_products_UKPF  = retype_index(sale_finished_products_UKPF, 'int', 'Артикул')


# получил текущие даныные по current_month
curr_per_1_UKPF = get_current_data(per_1_UKPF, global_index, current_month)
curr_per_2_UKPF_meat = get_current_data(per_2_UKPF_meat, global_index, current_month)
curr_per_2_UKPF_product = get_current_data(per_2_UKPF_product, global_index, current_month)
curr_per_2_UKPF_losses = get_current_data(per_2_UKPF_losses, global_index, current_month)
curr_per_3_UKPF = get_current_data(per_3_UKPF, global_index, current_month)
curr_Production_output_UKPF = get_current_data(Production_output, global_index, current_month)
UIS_UKPF = get_current_data(UIS_UKPF, global_index, current_month)
sale_finished_products_UKPF = get_current_data(sale_finished_products_UKPF, global_index, current_month)

# отсекли только то что в соответствующих переделах из справочников mapping


In [ ]:
# Блок получения данных по mapping (Отсекаем только то что есть в mapping)

cons_1_per = curr_per_1_UKPF[curr_per_1_UKPF['Артикул'].isin(mapping[mapping['Мэппинг УКПФ 1 передел']['Артикул']!=0]['Мэппинг УКПФ 1 передел']['Артикул'])].groupby(['Артикул','Продукция']).sum().reset_index()
cons_2_per_meat = curr_per_2_UKPF_meat[curr_per_2_UKPF_meat['Артикул'].isin(mapping[mapping['Мэппинг УКПФ 2 передел']['Артикул']!=0]['Мэппинг УКПФ 2 передел']['Артикул'])].groupby(['Артикул','Продукция']).sum().reset_index()
cons_2_per_product = curr_per_2_UKPF_product[curr_per_2_UKPF_product['Артикул'].isin(mapping[mapping['Мэппинг УКПФ 2 передел']['Артикул']!=0]['Мэппинг УКПФ 2 передел']['Артикул'])].groupby(['Артикул','Продукция']).sum().reset_index()
cons_2_per_losses = curr_per_2_UKPF_losses[~curr_per_2_UKPF_losses['Артикул'].apply(lambda x: isinstance(x, str))]


In [ ]:
dict_chast = rev_to_dict(dict_keys(mapping)['Артикул'],dict_keys(mapping)['Часть'])

# Присваем части
cons_1_per['часть'] = cons_1_per.apply(lambda x: insert_vpr(x['Артикул'],dict_chast,'Часть') ,axis = 1).astype('str')
# 2 передел для анализа
cons_2_per_meat['часть'] = cons_2_per_meat.apply(lambda x: insert_vpr(x['Артикул'],dict_chast,'Часть') ,axis = 1).astype('str')
cons_2_per_product['часть'] = cons_2_per_product.apply(lambda x: insert_vpr(x['Артикул'],dict_chast,'Часть') ,axis = 1).astype('str')
cons_2_per_losses['часть'] = cons_2_per_losses.apply(lambda x: insert_vpr(x['Артикул'],dict_chast,'Часть') ,axis = 1).astype('str')
# Выпуск ГП для анализа
curr_Production_output_UKPF['часть'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_chast,'Часть') ,axis = 1).astype('str')
curr_per_3_UKPF['часть'] = curr_per_3_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_chast,'Часть') ,axis = 1).astype('str')
# 3 передел для анализа
curr_per_3_UKPF = get_type_product(curr_per_3_UKPF)



In [ ]:
# группировка по частям
# Первый передел по частям УКПФ
part_1_cons_1_per = cons_1_per[['часть',current_month]].groupby('часть').sum().reset_index()
# объем живка
V_zhivka = DATA_upload[DATA_upload['Наименование']=='Объем живка, кг'].loc[:,current_month].values[0] 
#объем первого передела
V_1_peredel = cons_1_per[current_month].sum() 
# С/с 1 передела, без амортизации
SS_1_zperedel_without_amort = DATA_upload[DATA_upload['Наименование']=='Сырье (живая птица) без амортизации'].loc[:,current_month].values[0]/1000 
# Средняя с/с 1 передела, без амортизации
Mean_SS_1_zperedel_without_amort  = SS_1_zperedel_without_amort/V_1_peredel * 1000

In [ ]:
part_1_cons_1_per['%%от тушки'] = part_1_cons_1_per.apply(lambda x: x[current_month]/V_zhivka,axis = 1)


dict_koef_ef = rev_to_dict(koef_ef[koef_ef['Признак']=='убой']['Продукция'],koef_ef[koef_ef['Признак']=='убой']['Коэффициент'])
part_1_cons_1_per['Доля стоимости 1 передела'] = part_1_cons_1_per.apply(lambda x: insert_vpr(x['часть'],dict_koef_ef,'Коэффициент') ,axis = 1).astype('str')

part_1_cons_1_per['усл.ед.'] = part_1_cons_1_per['Доля стоимости 1 передела'].astype('float64') * part_1_cons_1_per['%%от тушки']

part_1_cons_1_per['Распределение с/с 1 передела'] = part_1_cons_1_per.apply(lambda x: SS_1_zperedel_without_amort * x['усл.ед.'] / part_1_cons_1_per['усл.ед.'].sum() ,axis = 1)


part_1_cons_1_per['С/с 1 передела'] = part_1_cons_1_per.apply(lambda x: 0 if  x[current_month] ==0 else  x['Распределение с/с 1 передела'] / x[current_month] * 1000 ,axis = 1)


# Конец первого передела

In [ ]:
# Начало второго передела
part_1_cons_2_per_meat = cons_2_per_meat[['часть',current_month]].groupby('часть').sum().reset_index()

part_1_cons_2_per_product = cons_2_per_product[['часть',current_month]].groupby('часть').sum().reset_index()

#Объем на разделку без потерь
V_raz_without_losses = part_1_cons_2_per_meat[current_month].sum() 


# С/с тушки, без амортизации

SS_tushki_without_amort = part_1_cons_1_per[part_1_cons_1_per['часть']=='Тушка']['С/с 1 передела'].values[0]

# С/с 2 передела, без амортизации
Mean_SS_1_zperedel_without_amort = V_raz_without_losses * SS_tushki_without_amort / 1000

part_1_cons_2_per_product['%%от тушки'] = part_1_cons_2_per_product.apply(lambda x: x[current_month]/V_raz_without_losses,axis = 1)



dict_koef_ef = rev_to_dict(koef_ef[koef_ef['Признак']=='обвалка']['Продукция'],koef_ef[koef_ef['Признак']=='обвалка']['Коэффициент'])
part_1_cons_2_per_product['Доля стоимости 2 передела'] = part_1_cons_2_per_product.apply(lambda x: insert_vpr(x['часть'],dict_koef_ef,'Коэффициент') ,axis = 1).astype('str')


part_1_cons_2_per_product['усл.ед.'] = part_1_cons_2_per_product['Доля стоимости 2 передела'].astype('float64') * part_1_cons_2_per_product['%%от тушки']

part_1_cons_2_per_product['Распределение с/с 2 передела'] = part_1_cons_2_per_product.apply(lambda x: Mean_SS_1_zperedel_without_amort * x['усл.ед.'] / part_1_cons_2_per_product['усл.ед.'].sum() ,axis = 1)

part_1_cons_2_per_product['С/с 2 передела'] = part_1_cons_2_per_product.apply(lambda x: 0 if  x[current_month] ==0 else  x['Распределение с/с 2 передела'] / x[current_month] * 1000 ,axis = 1)


In [ ]:
# ПГ_CC МП

def func(x, col, per1, per2_1, per2_2, per2_3, per3):
    filter_ = 'Артикул'
    sum_ = per1[per1[filter_] == x[filter_]][col].sum() + \
        per2_1[per2_1[filter_] == x[filter_]][col].sum() + \
        per2_2[per2_2[filter_] == x[filter_]][col].sum()+ \
        per2_3[per2_3[filter_] == x[filter_]][col].sum()+ \
        per3[(per3[filter_] == x[filter_])&(per3['Тип продукции'] == 'основная продукция')][col].sum()
    return sum_ if sum_>0 else x[col]
      

curr_Production_output_UKPF['Объем мяса  (без воды и специй),кг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,current_month, cons_1_per, cons_2_per_meat, cons_2_per_product, \
cons_2_per_losses, curr_per_3_UKPF) ,axis = 1)

# ------------------------------

df_sebes_group = part_1_cons_2_per_product[['часть','С/с 2 передела']].rename(columns={'С/с 2 передела': 'Себес'}).\
append( part_1_cons_1_per[['часть','С/с 1 передела']].rename(columns={'С/с 1 передела': 'Себес'}) ).\
groupby('часть').sum().reset_index()

dict_sebes = rev_to_dict(df_sebes_group['часть'],df_sebes_group['Себес'])
curr_Production_output_UKPF['Мясосырье, Птицевосдтво, тг/кг'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['часть'],dict_sebes,'Себес') ,axis = 1)

# ------------------------------

curr_Production_output_UKPF['C/c мясосырья, Птицеводство тыс. тг'] = curr_Production_output_UKPF.apply(lambda x: x['Объем мяса  (без воды и специй),кг'] * \
                                 x['Мясосырье, Птицевосдтво, тг/кг'] / 1000 ,axis = 1)

# ------------------------------

curr_Production_output_UKPF['C/c мясосырья, Птицеводство тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['C/c мясосырья, Птицеводство тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)

# ------------------------------

def func(x, col, uis, mask):
    filter_ = 'Артикул'
    sum_ = uis[(uis[filter_] == x[filter_]) & (uis['Номенклатура'] == mask)][col].sum()
    return sum_/1000
      
curr_Production_output_UKPF['Специи, добавки тыс. тг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,current_month, UIS_UKPF ,'Специи, добавки') ,axis = 1)

# ------------------------------

curr_Production_output_UKPF['Специи, добавки тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Специи, добавки тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)

# ------------------------------

curr_Production_output_UKPF['Упаковочный материал тыс. тг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,current_month, UIS_UKPF, 'Упаковочный материал') ,axis = 1)


# ------------------------------

curr_Production_output_UKPF['Упаковочный материал тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Упаковочный материал тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)

# ------------------------------

curr_Production_output_UKPF['Итого с/c с прямыми расходами тыс. тг'] = curr_Production_output_UKPF['C/c мясосырья, Птицеводство тыс. тг'] + \
curr_Production_output_UKPF['Специи, добавки тыс. тг'] + \
curr_Production_output_UKPF['Упаковочный материал тыс. тг']

# ------------------------------

curr_Production_output_UKPF['Итого с/c с прямыми расходами тг/кг'] = curr_Production_output_UKPF['C/c мясосырья, Птицеводство тг/кг'] + \
curr_Production_output_UKPF['Специи, добавки тг/кг'] + \
curr_Production_output_UKPF['Упаковочный материал тг/кг']

# ------------------------------

dict_slaughter_costs = rev_to_dict(mapping['Мэппинг УКПФ Затраты']['Артикул'],mapping['Мэппинг УКПФ Затраты']['Upstream'])
curr_Production_output_UKPF['Затраты на убой и потрошение, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_slaughter_costs,'Upstream') ,axis = 1).astype('str')

# ------------------------------

def func(x, col, df, df1, filter_):
    sum_ = df[df[filter_] == "да"][col].sum()
    
    if x[filter_] == 'да':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['upstream']['УиП']['всего'].sum()/1000
    else:
        return 0
    return 
        
      
curr_Production_output_UKPF['Затраты на убой и потрошение тыс. тг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,current_month, curr_Production_output_UKPF, NZ_UKPF,'Затраты на убой и потрошение, признак') ,axis = 1)

# ------------------------------

curr_Production_output_UKPF['Затраты на убой и потрошение тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на убой и потрошение тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)

# ------------------------------

dict_cutting_up= rev_to_dict(mapping['Мэппинг УКПФ Затраты']['Артикул'],mapping['Мэппинг УКПФ Затраты']['Разделка'])
curr_Production_output_UKPF['Затраты на разделку, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_cutting_up,'Разделка') ,axis = 1).astype('str')

# ------------------------------

def func(x, col, df, df1, filter_):
    sum_ = df[df[filter_] == "да"][col].sum()
    
    if x[filter_] == 'да':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['ОиР']['остальное'].sum()/1000
    elif x[filter_] == 'KFC':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['ОиР']['KFC'].sum()/1000  
    else:
        return 0
        
      
curr_Production_output_UKPF['Затраты на разделку тыс. тг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,
                                                current_month,
                                                curr_Production_output_UKPF,
                                                NZ_UKPF,
                                                'Затраты на убой и потрошение, признак'
                                               ) ,axis = 1)

# -------------------------------

curr_Production_output_UKPF['Затраты на разделку тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на разделку тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)

# -------------------------------

dict_cooling_down= rev_to_dict(mapping['Мэппинг УКПФ Затраты']['Артикул'],mapping['Мэппинг УКПФ Затраты']['Охлаждение'])
curr_Production_output_UKPF['Затраты на охлаждение, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_cooling_down,'Охлаждение') ,axis = 1).astype('str')

# -------------------------------

def func(x, col, df, df1, filter_):
    sum_ = df[df[filter_] == "да"][col].sum()
    
    if x[filter_] == 'да':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['охлаждение']['охл'].sum()/1000
    else:
        return 0
    return 
        
      
curr_Production_output_UKPF['Затраты на охлаждение тыс. тг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,
                                                current_month,
                                                curr_Production_output_UKPF,
                                                NZ_UKPF,
                                                'Затраты на охлаждение, признак'
                                               ) ,axis = 1)

# -------------------------------

curr_Production_output_UKPF['Затраты на охлаждение тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на охлаждение тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)

# -------------------------------

dict_freezing= rev_to_dict(mapping['Мэппинг УКПФ Затраты']['Артикул'],mapping['Мэппинг УКПФ Затраты']['Заморозка'])
curr_Production_output_UKPF['Затраты на заморозку, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_freezing,'Заморозка') ,axis = 1).astype('str')

# -------------------------------

def func(x, col, df, df1, filter_):
    sum_ = df[df[filter_] == "да"][col].sum()
    
    if x[filter_] == 'да':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['охлаждение']['зам'].sum()/1000
    else:
        return 0
    return 
        
      
curr_Production_output_UKPF['Затраты на заморозку тыс. тг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,
                                                current_month,
                                                curr_Production_output_UKPF,
                                                NZ_UKPF,
                                                'Затраты на заморозку, признак'
                                               ) ,axis = 1)

# --------------------------------

curr_Production_output_UKPF['Затраты на заморозку тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на заморозку тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)

# --------------------------------

dict_individual_pacts= rev_to_dict(mapping['Мэппинг УКПФ Затраты']['Артикул'],mapping['Мэппинг УКПФ Затраты']['индив. пакет'])
curr_Production_output_UKPF['Затраты на Индив. пакет, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_individual_pacts,'индив. пакет') ,axis = 1).astype('str')

# --------------------------------

dict_background= rev_to_dict(mapping['Мэппинг УКПФ Затраты']['Артикул'],mapping['Мэппинг УКПФ Затраты']['подложка'])
curr_Production_output_UKPF['Затраты на Подложку, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_background,'подложка') ,axis = 1).astype('str')

# --------------------------------

dict_group_package= rev_to_dict(mapping['Мэппинг УКПФ Затраты']['Артикул'],mapping['Мэппинг УКПФ Затраты']['групповой пакет'])
curr_Production_output_UKPF['Затраты на Групп. Пакет, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_group_package,'групповой пакет') ,axis = 1).astype('str')

# --------------------------------


NZ_UKPF = input_proc_packaging(curr_Production_output_UKPF,
                     ['Затраты на Индив. пакет, признак', \
                      'Затраты на Подложку, признак', \
                      'Затраты на Групп. Пакет, признак'],
                     current_month,
                     'Норма упаковки, кг/чел*час',
                     NZ_UKPF,
                     ['индив. пакет', 'подложка', 'групповой пакет'],
                     'Участок фасовки и упаковки '
                    )

# --------------------------------

def func(x, col, df, df1, filter_):
    sum_ = df[df[filter_] == "да"][col].sum()
    
    if x[filter_] == 'да':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Упак']['индив. пакет'].sum()/1000
    else:
        return 0
    return 
        
      
curr_Production_output_UKPF['Затраты на Индив. пакет тыс. тг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,
                                                current_month,
                                                curr_Production_output_UKPF,
                                                NZ_UKPF,
                                                'Затраты на Индив. пакет, признак'
                                               ) ,axis = 1)

# --------------------------------

curr_Production_output_UKPF['Затраты на Индив. пакет тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на Индив. пакет тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)


# --------------------------------

def func(x, col, df, df1, filter_):
    sum_ = df[df[filter_] == "да"][col].sum()
    
    if x[filter_] == 'да':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Упак']['подложка'].sum()/1000
    else:
        return 0
    return 
        
      
curr_Production_output_UKPF['Затраты на Подложку тыс. тг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,
                                                current_month,
                                                curr_Production_output_UKPF,
                                                NZ_UKPF,
                                                'Затраты на Подложку, признак'
                                               ) ,axis = 1)

# --------------------------------

curr_Production_output_UKPF['Затраты на Подложку тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на Подложку тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)


# --------------------------------

def func(x, col, df, df1, filter_):
    sum_ = df[df[filter_] == "да"][col].sum()
    
    if x[filter_] == 'да':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Упак']['групповой пакет'].sum()/1000
    else:
        return 0
    return 
        
      
curr_Production_output_UKPF['Затраты на Групп. Пакет тыс. тг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,
                                                current_month,
                                                curr_Production_output_UKPF,
                                                NZ_UKPF,
                                                'Затраты на Групп. Пакет, признак'
                                               ) ,axis = 1)

# --------------------------------

curr_Production_output_UKPF['Затраты на Групп. Пакет тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на Групп. Пакет тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)

# --------------------------------

dict_marinate= rev_to_dict(mapping['Мэппинг УКПФ Затраты']['Артикул'],mapping['Мэппинг УКПФ Затраты']['маринация'])
curr_Production_output_UKPF['Затраты на Маринацию, признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_marinate,'маринация') ,axis = 1).astype('str')

# --------------------------------

def func(x, col, df, df1, filter_):
    sum_ = df[df[filter_] == "да"][col].sum()
    
    if x[filter_] == 'да':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Маринады']['остальное'].sum()/1000
    elif x[filter_] == 'KFC':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Маринады']['KFC'].sum()/1000  
    else:
        return 0
        
      
curr_Production_output_UKPF['Затраты на Маринацию тыс. тг'] = \
curr_Production_output_UKPF.apply(lambda x:func(x,
                                                current_month,
                                                curr_Production_output_UKPF,
                                                NZ_UKPF,
                                                'Затраты на Маринацию, признак'
                                               ) ,axis = 1)

# --------------------------------

curr_Production_output_UKPF['Затраты на Маринацию тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на Маринацию тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)

# --------------------------------

dict_pressing= rev_to_dict(mapping['Мэппинг УКПФ Затраты']['Артикул'],mapping['Мэппинг УКПФ Затраты']['Прессование'])
curr_Production_output_UKPF['Затраты на Прессование признак'] = curr_Production_output_UKPF.apply(lambda x: insert_vpr(x['Артикул'],dict_pressing,'Прессование') ,axis = 1).astype('str')

# --------------------------------

def func(x, col, df, df1, filter_):
    sum_ = df[df[filter_] == "да"][col].sum()
    
    if x[filter_] == 'да':
        return (x[current_month]/sum_) * df1[df1['Месяц']['Месяц']['Месяц']==current_month]['downstream']['Упак']['групповой пакет'].sum()/1000
    else:
        return 0
    return 
        
      
curr_Production_output_UKPF['Затраты на Прессование тыс. тг'] = 0
curr_Production_output_UKPF.apply(lambda x:func(x,
                                                current_month,
                                                curr_Production_output_UKPF,
                                                NZ_UKPF,
                                                'Затраты на Групп. Пакет, признак'
                                               ) ,axis = 1)


# --------------------------------


curr_Production_output_UKPF['Затраты на Прессование тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Затраты на Прессование тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)


# --------------------------------

curr_Production_output_UKPF['Итого накладная с/с тыс. тг'] =curr_Production_output_UKPF['Затраты на убой и потрошение тыс. тг']+ \
                                                            curr_Production_output_UKPF['Затраты на охлаждение тыс. тг']+ \
                                                            curr_Production_output_UKPF['Затраты на заморозку тыс. тг']+ \
                                                            curr_Production_output_UKPF['Затраты на Индив. пакет тыс. тг']+ \
                                                            curr_Production_output_UKPF['Затраты на Групп. Пакет тыс. тг']+ \
                                                            curr_Production_output_UKPF['Затраты на Маринацию тыс. тг']+ \
                                                            curr_Production_output_UKPF['Затраты на Прессование тыс. тг']

curr_Production_output_UKPF['Итого накладная с/с тг/кг'] =curr_Production_output_UKPF['Затраты на убой и потрошение тг/кг']+ \
                                                            curr_Production_output_UKPF['Затраты на охлаждение тг/кг']+ \
                                                            curr_Production_output_UKPF['Затраты на заморозку тг/кг']+ \
                                                            curr_Production_output_UKPF['Затраты на Индив. пакет тг/кг']+ \
                                                            curr_Production_output_UKPF['Затраты на Групп. Пакет тг/кг']+ \
                                                            curr_Production_output_UKPF['Затраты на Маринацию тг/кг']+ \
                                                            curr_Production_output_UKPF['Затраты на Прессование тг/кг']


curr_Production_output_UKPF['Итого с/c тыс. тг'] = curr_Production_output_UKPF['Итого с/c с прямыми расходами тыс. тг'] + \
                                                    curr_Production_output_UKPF['Итого накладная с/с тыс. тг']


curr_Production_output_UKPF['Итого с/c тг/кг'] = curr_Production_output_UKPF.apply(lambda x: 0 if x[current_month]==0 else x['Итого с/c тыс. тг'] / \
                                 x[current_month] * 1000 ,axis = 1)

In [ ]:
# таблица с остатками

start_date = datetime.datetime(2021, 1, 1, 0, 0)

cols_join = [x for x in curr_Production_output_UKPF.rename(columns={current_month: 'Объем кг'}).columns if x.find('тг/кг')!=-1]

cols_join.extend(['Объем кг'])

# ost_start = ost_start[start_date]

# блок формирования левой части остатков
naim = ('Продукция','Продукция')
art = ('Артикул','Артикул')

f_prt = ost_start[[naim,art]]
f_prt.columns = ['Продукция','Артикул']
# общий список из остатков , производства и продаж гп
arr_ost = f_prt.append(curr_Production_output_UKPF[['Артикул','Продукция']]).append(sale_finished_products_UKPF[['Продукция','Артикул']]).drop_duplicates(subset=['Артикул'])

arr_ost = arr_ost.set_index('Артикул')
curr_Production_output_UKPF = curr_Production_output_UKPF.set_index('Артикул')

arr_ost = arr_ost[~arr_ost.index.duplicated(keep='first')]


# Берем начальные остатки
ost_start.columns = [x[1] for x in ost_start.columns]
ost_start = ost_start.set_index('Артикул')


cons_ost_start = pd.concat([ost_start[cols_join],arr_ost] ,axis = 1)

ml = pd.MultiIndex.from_tuples([ ('Остаток',start_date, x) for x in  cons_ost_start.columns])
cons_ost_start.columns =ml



In [ ]:
# тут формируем циклом приход расход остаток

# приход для объединения с остатками
production_for_join_ost = curr_Production_output_UKPF.rename(columns={current_month: 'Объем кг'})[cols_join]


ml = pd.MultiIndex.from_tuples([ ('Приход',current_month, x) for x in  production_for_join_ost.columns])
production_for_join_ost.columns =ml

it = pd.concat([cons_ost_start,production_for_join_ost] ,axis = 1)

In [ ]:
sale_finished_products_UKPF

In [ ]:
curr_Production_output_UKPF

In [ ]:
ost_start

In [ ]:
curr_Production_output_UKPF[cols_join]

In [ ]:

pd.concat([arr_ost,curr_Production_output_UKPF[cols_join]] ,axis = 1)

In [ ]:
ost_start.columns = [x[1] for x in ost_start.columns]

In [ ]:
ost_start.set_index('Артикул')

In [ ]:
curr_Production_output_UKPF

In [ ]:
ost_start

In [ ]:
cols_join

In [ ]:
NZ_UKPF =  pd.read_excel(directory+'УКПФ Формы маржинальности Прогноз 2021.xlsx',sheet_name='Накладные затраты',header=[0,1,2])

In [ ]:
NZ_UKPF[NZ_UKPF['Месяц']['Месяц']['Месяц']==current_month]['downstream']['ОиР']['KFC']

In [ ]:
curr_Production_output_UKPF[curr_Production_output_UKPF['Затраты на Подложку, признак'] == "да"][current_month].sum()

In [ ]:
curr_Production_output_UKPF[curr_Production_output_UKPF['Затраты на Групп. Пакет, признак'] == "да"][current_month].sum()

In [ ]:
NZ_UKPF[NZ_UKPF['Наименование']['Наименование']['Наименование']=='Норма упаковки, кг/чел*час'] \
['downstream']['Упак']['групповой пакет'].sum()

In [ ]:
NZ_UKPF[NZ_UKPF['Наименование']['Наименование']['Наименование']=='Норма упаковки, кг/чел*час'] \
['downstream']['Упак']['подложка'].sum()

In [ ]:
NZ_UKPF[NZ_UKPF['Наименование']['Наименование']['Наименование']=='Норма упаковки, кг/чел*час'] \
['downstream']['Упак']['индив. пакет'].sum()

In [ ]:
NZ_UKPF.loc[NZ_UKPF['Месяц']['Месяц']['Месяц']==current_month ]['downstream']['Упак']['всего'].sum()

In [ ]:
NZ_UKPF.loc[NZ_UKPF['Месяц']['Месяц']['Месяц']==current_month,(cols1)]

In [ ]:
a.append(1)

In [ ]:
a[-1] = 5

In [ ]:
# Установка новых % в накладные расходы по упаковке
def input_proc_packaging(df,arr_priznak, date, filter_, df2,  arr_priznak1 ,filter_2):
    cols1 = ('downstream', 'Упак', arr_priznak1[0])
    cols2 = ('downstream', 'Упак', arr_priznak1[1])
    cols3 = ('downstream', 'Упак', arr_priznak1[2])
    
    d1 = df[df[arr_priznak[0]] == "да"][current_month].sum() / \
    df2[df2['Наименование']['Наименование']['Наименование']==filter_].loc[:,cols1].sum()
    
    d2 = df[df[arr_priznak[1]] == "да"][current_month].sum() / \
    df2[df2['Наименование']['Наименование']['Наименование']==filter_].loc[:,cols2].sum()
    
    d3 = df[df[arr_priznak[2]] == "да"][current_month].sum() / \
    df2[df2['Наименование']['Наименование']['Наименование']==filter_].loc[:,cols3].sum()
    
    df2.loc[df2['Наименование']['Наименование']['Наименование']==filter_2,(cols1)] = d1/(d1+d2+d3)
    df2.loc[df2['Наименование']['Наименование']['Наименование']==filter_2,(cols2)] = d2/(d1+d2+d3)
    df2.loc[df2['Наименование']['Наименование']['Наименование']==filter_2,(cols3)] = d3/(d1+d2+d3)
    
    return df2

In [ ]:
NZ_UKPF[NZ_UKPF['Наименование']['Наименование']['Наименование']=='Участок фасовки и упаковки '].loc[:,('downstream', 'Упак', 'подложка')]

In [ ]:
NZ_UKPF = input_proc_packaging(curr_Production_output_UKPF,
                     ['Затраты на Индив. пакет, признак', \
                      'Затраты на Подложку, признак', \
                      'Затраты на Групп. Пакет, признак'],
                     current_month,
                     'Норма упаковки, кг/чел*час',
                     NZ_UKPF,
                     ['индив. пакет', 'подложка', 'групповой пакет'],
                     'Участок фасовки и упаковки '
                    )

In [ ]:
['Наименование']*3

In [ ]:
NZ_UKPF.loc[NZ_UKPF['Наименование']['Наименование']['Наименование']=='Затраты на Индив. пакет, признак']

In [ ]:
a = ('Наименование','Наименование','Наименование')
NZ_UKPF.loc[NZ_UKPF[a]=='Норма упаковки, кг/чел*час',:]

In [ ]:
NZ_UKPF['Наименование']['Наименование']['Наименование']

In [ ]:
part_1_cons_2_per_product

In [ ]:
NZ_UKPF.columns

In [ ]:
pd.MultiIndex.from_arrays([1,2,3],[1,2,3])

In [ ]:
arr_ost.columns

In [ ]:
arr_ost.columns

In [ ]:
a = pd.DataFrame({'a':[1,2,3,5]})
b = pd.DataFrame({'a':[1,2,3],'b':[6,7,8]})

In [ ]:
pd.concat([a.set_index('a'),b.set_index('a')],axis = 1)

In [ ]:
a.index

In [ ]:
len(set(list(arr_ost.index)))

In [ ]:
a = curr_Production_output_UKPF.set_index('Артикул').index

In [ ]:
len(list(arr_ost.index))

In [ ]:
ost_start[start_date]

In [ ]:
arr_ost[~arr_ost.index.duplicated(keep='first')]

In [ ]:
cons_ost_start.to_excel(directory+'ddd.xlsx')

In [ ]:
cons_ost_start = pd.concat([ost_start[cols_join],arr_ost] ,axis = 1)

In [ ]:
pd.read_excel(directory+'ddd.xlsx',header=[0,1],index_col=0)

In [ ]:
curr_Production_output_UKPF